# STM Transit Delay Data Preparation

## Data description

### Real-time STM Trip Updates

`current_time` timestamp of the time the data was collected<br>
`trip_id` unique identifier of a trip<br>
`route_id` bus or metro line<br>
`start_date` schedule date<br>
`stop_id` stop number<br>
`arrival_time` actual arrival time, in milliseconds<br>
`departure_time` actual departure time, in milliseconds<br>
`schedule_relationship` state of the trip, 0 means scheduled and 1 means skipped

### Scheduled STM Trips

`trip_id` unique identifier of a trip<br>
`arrival_time` scheduled arrival time, in milliseconds<br>
`departure_time` scheduled departure time, in milliseconds<br>
`stop_id` stop number<br>
`stop_sequence` sequence of the stop, for ordering

### STM Stops

`stop_id` unique identifier of a stop<br>
`stop_code` stop number<br>
`stop_name` stop name<br>
`stop_lat` stop latitude<br>
`stop_lon` stop longitude<br>
`stop_url` stop web page<br>
`location_type` stop type, 1 being a metro station and 2 a bus stop<br>
`parent_station` parent station (ex: a metro station with multiple exits)<br>
`wheelchair_boarding` indicates if the stop is accessible for people in wheelchair, 1 being true and 2 being false

### Weather Archive

`time` date and hour or the archived weather<br>
`temperature` air temperature at 2 meters above ground, in Celsius<br>
`precipitation` total precipitation (rain, showers, snow) sum of the preceding hour, in millimeters<br>
`windspeed` wind speed at 10 meters above ground, in km/h<br>
`weathercode` World Meteorological Organization (WMO) code

## Imports

In [30]:
from datetime import timedelta
import numpy as np
import pandas as pd

In [31]:
# Set timezone
local_timezone = 'Canada/Eastern'

## Load Data

In [32]:
real_stm_df = pd.read_csv('../data/fetched_stm.csv', low_memory=False)
real_stm_df.head()

,current_time,trip_id,route_id,start_date,stop_id,arrival_time,departure_time,schedule_relationship
0,1.745385e+09,285028348,189,20250422,54433,1745384718,1745384718,0
1,1.745385e+09,285028348,189,20250422,54444,1745384751,1745384751,0
2,1.745385e+09,285028348,189,20250422,54445,1745384785,1745384785,0
3,1.745385e+09,285028348,189,20250422,54451,1745384806,1745384806,0
4,1.745385e+09,285028348,189,20250422,54456,1745384829,1745384829,0


In [33]:
planned_stm_df = pd.read_csv('../data/stop_times_2025-04-23.txt')
planned_stm_df.head()

,trip_id,arrival_time,departure_time,stop_id,stop_sequence
0,281570788,05:58:00,05:58:00,51095,1
1,281570788,05:59:39,05:59:39,51126,2
2,281570788,06:00:06,06:00:06,51113,3
3,281570788,06:00:44,06:00:44,51084,4
4,281570788,06:01:17,06:01:17,51063,5


In [34]:
stops_df = pd.read_csv('../data/stops_2025-04-23.txt')
stops_df.head()

,stop_id,stop_code,stop_name,stop_lat,stop_lon,stop_url,location_type,parent_station,wheelchair_boarding
0,STATION_M118,10118,STATION ANGRIGNON,45.446466,-73.603118,NaN,1,NaN,1
1,43,10118,Station Angrignon,45.446466,-73.603118,http://www.stm.info/fr/infos/reseaux/metro/ang...,0,STATION_M118,1
2,43-01,10118,Station Angrignon,45.446319,-73.603835,NaN,2,STATION_M118,1
3,STATION_M120,10120,STATION MONK,45.451158,-73.593242,NaN,1,NaN,2
4,42,10120,Station Monk,45.451158,-73.593242,http://www.stm.info/fr/infos/reseaux/metro/monk,0,STATION_M120,2


In [35]:
weather_df = pd.read_csv('../data/fetched_weather.csv')
weather_df.head()

,time,temperature,precipitation,windspeed,weathercode
0,2025-04-20T00:00,10.9,0.0,21.1,3
1,2025-04-20T01:00,6.9,0.0,21.9,2
2,2025-04-20T02:00,5.1,0.0,16.3,1
3,2025-04-20T03:00,3.7,0.0,16.1,0
4,2025-04-20T04:00,2.5,0.0,16.3,0


## Merge Data

### Realtime and Scheduled Trips

In [36]:
stm_trips_df = pd.merge(left=real_stm_df, right=planned_stm_df, how='inner', on=['trip_id', 'stop_id'])
stm_trips_df.head()

,current_time,trip_id,route_id,start_date,stop_id,arrival_time_x,departure_time_x,schedule_relationship,arrival_time_y,departure_time_y,stop_sequence
0,1.745385e+09,285028348,189,20250422,54433,1745384718,1745384718,0,25:05:08,25:05:08,20
1,1.745385e+09,285028348,189,20250422,54444,1745384751,1745384751,0,25:05:51,25:05:51,21
2,1.745385e+09,285028348,189,20250422,54445,1745384785,1745384785,0,25:06:25,25:06:25,22
3,1.745385e+09,285028348,189,20250422,54451,1745384806,1745384806,0,25:06:46,25:06:46,23
4,1.745385e+09,285028348,189,20250422,54456,1745384829,1745384829,0,25:07:09,25:07:09,24


In [37]:
stm_trips_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1455245 entries, 0 to 1455244
Data columns (total 11 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   current_time           1455245 non-null  float64
 1   trip_id                1455245 non-null  int64  
 2   route_id               1455245 non-null  object 
 3   start_date             1455245 non-null  int64  
 4   stop_id                1455245 non-null  int64  
 5   arrival_time_x         1455245 non-null  int64  
 6   departure_time_x       1455245 non-null  int64  
 7   schedule_relationship  1455245 non-null  int64  
 8   arrival_time_y         1455245 non-null  object 
 9   departure_time_y       1455245 non-null  object 
 10  stop_sequence          1455245 non-null  int64  
dtypes: float64(1), int64(7), object(3)
memory usage: 122.1+ MB


In [38]:
# Convert start_date to datetime
stm_trips_df['start_date'] = pd.to_datetime(stm_trips_df['start_date'], format='%Y%m%d')
stm_trips_df.head()

,current_time,trip_id,route_id,start_date,stop_id,arrival_time_x,departure_time_x,schedule_relationship,arrival_time_y,departure_time_y,stop_sequence
0,1.745385e+09,285028348,189,2025-04-22,54433,1745384718,1745384718,0,25:05:08,25:05:08,20
1,1.745385e+09,285028348,189,2025-04-22,54444,1745384751,1745384751,0,25:05:51,25:05:51,21
2,1.745385e+09,285028348,189,2025-04-22,54445,1745384785,1745384785,0,25:06:25,25:06:25,22
3,1.745385e+09,285028348,189,2025-04-22,54451,1745384806,1745384806,0,25:06:46,25:06:46,23
4,1.745385e+09,285028348,189,2025-04-22,54456,1745384829,1745384829,0,25:07:09,25:07:09,24


In [39]:
def parse_gtfs_time(row) -> pd.Timestamp:
	'''
	Converts GTFS time string (e.g., '25:30:00') to datetime
	based on the arrival time.
	'''
	hours, minutes, seconds = map(int, row['arrival_time_y'].split(':'))
	total_seconds = hours * 3600 + minutes * 60 + seconds

	parsed_time = row['start_date'] + timedelta(seconds=total_seconds)
	return parsed_time

In [40]:
# Convert planned arrival time to localized datetime
stm_trips_df['scheduled_arrival_time'] = stm_trips_df.apply(parse_gtfs_time, axis=1)
stm_trips_df['scheduled_arrival_time'] = stm_trips_df['scheduled_arrival_time'].dt.tz_localize(local_timezone)
stm_trips_df.head()

,current_time,trip_id,route_id,start_date,stop_id,arrival_time_x,departure_time_x,schedule_relationship,arrival_time_y,departure_time_y,stop_sequence,scheduled_arrival_time
0,1.745385e+09,285028348,189,2025-04-22,54433,1745384718,1745384718,0,25:05:08,25:05:08,20,2025-04-23 01:05:08-04:00
1,1.745385e+09,285028348,189,2025-04-22,54444,1745384751,1745384751,0,25:05:51,25:05:51,21,2025-04-23 01:05:51-04:00
2,1.745385e+09,285028348,189,2025-04-22,54445,1745384785,1745384785,0,25:06:25,25:06:25,22,2025-04-23 01:06:25-04:00
3,1.745385e+09,285028348,189,2025-04-22,54451,1745384806,1745384806,0,25:06:46,25:06:46,23,2025-04-23 01:06:46-04:00
4,1.745385e+09,285028348,189,2025-04-22,54456,1745384829,1745384829,0,25:07:09,25:07:09,24,2025-04-23 01:07:09-04:00


In [41]:
# Convert planned time to timestamp in milliseconds since epoch
stm_trips_df['scheduled_arrival_time'] = stm_trips_df['scheduled_arrival_time'].astype('int64') // 10**6
stm_trips_df.head()

,current_time,trip_id,route_id,start_date,stop_id,arrival_time_x,departure_time_x,schedule_relationship,arrival_time_y,departure_time_y,stop_sequence,scheduled_arrival_time
0,1.745385e+09,285028348,189,2025-04-22,54433,1745384718,1745384718,0,25:05:08,25:05:08,20,1745384708000
1,1.745385e+09,285028348,189,2025-04-22,54444,1745384751,1745384751,0,25:05:51,25:05:51,21,1745384751000
2,1.745385e+09,285028348,189,2025-04-22,54445,1745384785,1745384785,0,25:06:25,25:06:25,22,1745384785000
3,1.745385e+09,285028348,189,2025-04-22,54451,1745384806,1745384806,0,25:06:46,25:06:46,23,1745384806000
4,1.745385e+09,285028348,189,2025-04-22,54456,1745384829,1745384829,0,25:07:09,25:07:09,24,1745384829000


In [42]:
# Convert realtime arrival and departure time to milliseconds
stm_trips_df['arrival_time_x'] = stm_trips_df['arrival_time_x'] * 1000
stm_trips_df['departure_time_x'] = stm_trips_df['departure_time_x'] * 1000

In [43]:
# Analyse distribution
stm_trips_df[['arrival_time_x', 'departure_time_x']].describe()

,arrival_time_x,departure_time_x
count,1.455245e+06,1.455245e+06
mean,1.660729e+12,1.640413e+12
std,3.752157e+11,4.152009e+11
min,0.000000e+00,0.000000e+00
25%,1.745451e+12,1.745449e+12
50%,1.745509e+12,1.745508e+12
75%,1.745538e+12,1.745538e+12
max,1.745595e+12,1.745595e+12


In [45]:
# Replace null arrival time by departure time, as they are usually the same
zero_mask = stm_trips_df['arrival_time_x'] == 0
stm_trips_df.loc[zero_mask, 'arrival_time_x'] = stm_trips_df.loc[zero_mask, 'departure_time_x']

In [47]:
# Delete the rows with null arrival times
zero_mask = stm_trips_df['arrival_time_x'] == 0
stm_trips_df = stm_trips_df[~zero_mask]

In [49]:
# Rename real time arrival time
stm_trips_df = stm_trips_df.rename(columns={'arrival_time_x': 'realtime_arrival_time'})
stm_trips_df.head()

,current_time,trip_id,route_id,start_date,stop_id,realtime_arrival_time,departure_time_x,schedule_relationship,arrival_time_y,departure_time_y,stop_sequence,scheduled_arrival_time
0,1.745385e+09,285028348,189,2025-04-22,54433,1745384718000,1745384718000,0,25:05:08,25:05:08,20,1745384708000
1,1.745385e+09,285028348,189,2025-04-22,54444,1745384751000,1745384751000,0,25:05:51,25:05:51,21,1745384751000
2,1.745385e+09,285028348,189,2025-04-22,54445,1745384785000,1745384785000,0,25:06:25,25:06:25,22,1745384785000
3,1.745385e+09,285028348,189,2025-04-22,54451,1745384806000,1745384806000,0,25:06:46,25:06:46,23,1745384806000
4,1.745385e+09,285028348,189,2025-04-22,54456,1745384829000,1745384829000,0,25:07:09,25:07:09,24,1745384829000


### Trips and Stops

In [50]:
# Merge stops to trips
merged_stm_df = pd.merge(left=stm_trips_df, right=stops_df, how='inner', left_on='stop_id', right_on='stop_code')
merged_stm_df.head()

,current_time,trip_id,route_id,start_date,stop_id_x,realtime_arrival_time,departure_time_x,schedule_relationship,arrival_time_y,departure_time_y,...,scheduled_arrival_time,stop_id_y,stop_code,stop_name,stop_lat,stop_lon,stop_url,location_type,parent_station,wheelchair_boarding
0,1.745385e+09,285028348,189,2025-04-22,54433,1745384718000,1745384718000,0,25:05:08,25:05:08,...,1745384708000,54433,54433,Notre-Dame / No 10150,45.617546,-73.507835,https://www.stm.info/fr/recherche#stq=54433,0,NaN,1
1,1.745385e+09,285028348,189,2025-04-22,54444,1745384751000,1745384751000,0,25:05:51,25:05:51,...,1745384751000,54444,54444,Notre-Dame / Gamble,45.621630,-73.505533,https://www.stm.info/fr/recherche#stq=54444,0,NaN,1
2,1.745385e+09,285028348,189,2025-04-22,54445,1745384785000,1745384785000,0,25:06:25,25:06:25,...,1745384785000,54445,54445,Notre-Dame / No 10800,45.624606,-73.503332,https://www.stm.info/fr/recherche#stq=54445,0,NaN,1
3,1.745385e+09,285028348,189,2025-04-22,54451,1745384806000,1745384806000,0,25:06:46,25:06:46,...,1745384806000,54451,54451,Notre-Dame / Richard,45.626270,-73.501486,https://www.stm.info/fr/recherche#stq=54451,0,NaN,1
4,1.745385e+09,285028348,189,2025-04-22,54456,1745384829000,1745384829000,0,25:07:09,25:07:09,...,1745384829000,54456,54456,Notre-Dame / Hinton,45.628078,-73.499449,https://www.stm.info/fr/recherche#stq=54456,0,NaN,1


In [51]:
# Keep relevant columns
merged_stm_df = merged_stm_df[[
  'trip_id',
  'route_id',
  'stop_id_x',
  'stop_lat',
  'stop_lon',
  'stop_sequence',
  'wheelchair_boarding',
  'realtime_arrival_time',
  'scheduled_arrival_time'
]]
merged_stm_df = merged_stm_df.rename(columns={'stop_id_x': 'stop_id'})
merged_stm_df.head()

,trip_id,route_id,stop_id,stop_lat,stop_lon,stop_sequence,wheelchair_boarding,realtime_arrival_time,scheduled_arrival_time
0,285028348,189,54433,45.617546,-73.507835,20,1,1745384718000,1745384708000
1,285028348,189,54444,45.621630,-73.505533,21,1,1745384751000,1745384751000
2,285028348,189,54445,45.624606,-73.503332,22,1,1745384785000,1745384785000
3,285028348,189,54451,45.626270,-73.501486,23,1,1745384806000,1745384806000
4,285028348,189,54456,45.628078,-73.499449,24,1,1745384829000,1745384829000


In [54]:
# Convert arrival timestamp to datetime
rt_arrival_dt = pd.to_datetime(merged_stm_df['realtime_arrival_time'], origin='unix', unit='ms', utc=True)
rt_arrival_dt

0         2025-04-23 05:05:18+00:00
1         2025-04-23 05:05:51+00:00
2         2025-04-23 05:06:25+00:00
3         2025-04-23 05:06:46+00:00
4         2025-04-23 05:07:09+00:00
                     ...           
1412687   2025-04-25 13:10:48+00:00
1412688   2025-04-25 13:11:34+00:00
1412689   2025-04-25 13:13:21+00:00
1412690   2025-04-25 13:14:00+00:00
1412691   2025-04-25 13:17:00+00:00
Name: realtime_arrival_time, Length: 1412692, dtype: datetime64[ns, UTC]

In [55]:
# TODO: remove this cell after collecting historical data
# Remove 3 days to match historical data
rt_arrival_dt = rt_arrival_dt - pd.DateOffset(days=3)
rt_arrival_dt

0         2025-04-20 05:05:18+00:00
1         2025-04-20 05:05:51+00:00
2         2025-04-20 05:06:25+00:00
3         2025-04-20 05:06:46+00:00
4         2025-04-20 05:07:09+00:00
                     ...           
1412687   2025-04-22 13:10:48+00:00
1412688   2025-04-22 13:11:34+00:00
1412689   2025-04-22 13:13:21+00:00
1412690   2025-04-22 13:14:00+00:00
1412691   2025-04-22 13:17:00+00:00
Name: realtime_arrival_time, Length: 1412692, dtype: datetime64[ns, UTC]

In [56]:
# Round arrival time to the nearest hour
rounded_arrival_dt = rt_arrival_dt.dt.round('h')
rounded_arrival_dt

0         2025-04-20 05:00:00+00:00
1         2025-04-20 05:00:00+00:00
2         2025-04-20 05:00:00+00:00
3         2025-04-20 05:00:00+00:00
4         2025-04-20 05:00:00+00:00
                     ...           
1412687   2025-04-22 13:00:00+00:00
1412688   2025-04-22 13:00:00+00:00
1412689   2025-04-22 13:00:00+00:00
1412690   2025-04-22 13:00:00+00:00
1412691   2025-04-22 13:00:00+00:00
Name: realtime_arrival_time, Length: 1412692, dtype: datetime64[ns, UTC]

In [57]:
# Format time to match weather data
merged_stm_df['time'] = rounded_arrival_dt.dt.strftime('%Y-%m-%dT%H:%M')
merged_stm_df.head()

,trip_id,route_id,stop_id,stop_lat,stop_lon,stop_sequence,wheelchair_boarding,realtime_arrival_time,scheduled_arrival_time,time
0,285028348,189,54433,45.617546,-73.507835,20,1,1745384718000,1745384708000,2025-04-20T05:00
1,285028348,189,54444,45.621630,-73.505533,21,1,1745384751000,1745384751000,2025-04-20T05:00
2,285028348,189,54445,45.624606,-73.503332,22,1,1745384785000,1745384785000,2025-04-20T05:00
3,285028348,189,54451,45.626270,-73.501486,23,1,1745384806000,1745384806000,2025-04-20T05:00
4,285028348,189,54456,45.628078,-73.499449,24,1,1745384829000,1745384829000,2025-04-20T05:00


In [58]:
# Get duplicates
duplicate_mask = merged_stm_df.duplicated()
merged_stm_df[duplicate_mask]

,trip_id,route_id,stop_id,stop_lat,stop_lon,stop_sequence,wheelchair_boarding,realtime_arrival_time,scheduled_arrival_time,time
253230,285285001,811,62138,45.589525,-73.537341,1,1,1745447520000,1745444760000,2025-04-20T23:00
253231,285285001,811,62138,45.589525,-73.537341,22,1,1745447520000,1745447520000,2025-04-20T23:00
258234,285285013,811,62138,45.589525,-73.537341,1,1,1745448180000,1745445480000,2025-04-20T23:00
258235,285285013,811,62138,45.589525,-73.537341,22,1,1745448180000,1745448180000,2025-04-20T23:00
297697,285007882,72,55717,45.508261,-73.672905,34,1,1745450840000,1745450840000,2025-04-20T23:00
...,...,...,...,...,...,...,...,...,...,...
1412687,284728098,95,52584,45.594369,-73.565162,39,1,1745586648000,1745586648000,2025-04-22T13:00
1412688,284728098,95,54697,45.595766,-73.564835,40,1,1745586694000,1745586694000,2025-04-22T13:00
1412689,284728098,95,54578,45.597480,-73.569943,41,1,1745586801000,1745586801000,2025-04-22T13:00
1412690,284728098,95,54582,45.598514,-73.569275,42,1,1745586840000,1745586840000,2025-04-22T13:00


In [59]:
# Remove duplicates
merged_stm_df = merged_stm_df.drop_duplicates()

### STM and Weather

In [60]:
# Merge STM data with weather data
df = pd.merge(left=merged_stm_df, right=weather_df, how='inner', on='time').drop('time', axis=1)
df.head()

,trip_id,route_id,stop_id,stop_lat,stop_lon,stop_sequence,wheelchair_boarding,realtime_arrival_time,scheduled_arrival_time,temperature,precipitation,windspeed,weathercode
0,285028348,189,54433,45.617546,-73.507835,20,1,1745384718000,1745384708000,1.5,0.0,16.2,0
1,285028348,189,54444,45.621630,-73.505533,21,1,1745384751000,1745384751000,1.5,0.0,16.2,0
2,285028348,189,54445,45.624606,-73.503332,22,1,1745384785000,1745384785000,1.5,0.0,16.2,0
3,285028348,189,54451,45.626270,-73.501486,23,1,1745384806000,1745384806000,1.5,0.0,16.2,0
4,285028348,189,54456,45.628078,-73.499449,24,1,1745384829000,1745384829000,1.5,0.0,16.2,0


## Prepare Data

In [61]:
# Convert real and scheduled timestamps
rt_arrival_dt = pd.to_datetime(df['realtime_arrival_time'], origin='unix', unit='ms', utc=True) # TODO: remove this and use previous variable
sch_arrival_dt = pd.to_datetime(df['scheduled_arrival_time'], origin='unix', unit='ms', utc=True)

In [62]:
# Calculate delay in seconds (real - scheduled)
df['delay'] = (rt_arrival_dt - sch_arrival_dt) / pd.Timedelta(seconds=1)

In [63]:
# Sort data
df = df.sort_values(by=['trip_id', 'stop_sequence'])

In [64]:
# Get previous stop delay
df['delay_previous_stop'] = df.groupby('trip_id')['delay'].shift(1)
df['delay_previous_stop'] = df['delay_previous_stop'].fillna(0)

In [65]:
# Convert route_id to integer
df['route_id'] = df['route_id'].astype('int64')

In [66]:
# Convert wheelchair_boarding to boolean
df['wheelchair_boarding'] = np.where(df['wheelchair_boarding'] == 1, True, False)

In [67]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1302608 entries, 26075 to 373539
Data columns (total 15 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   trip_id                 1302608 non-null  int64  
 1   route_id                1302608 non-null  int64  
 2   stop_id                 1302608 non-null  int64  
 3   stop_lat                1302608 non-null  float64
 4   stop_lon                1302608 non-null  float64
 5   stop_sequence           1302608 non-null  int64  
 6   wheelchair_boarding     1302608 non-null  bool   
 7   realtime_arrival_time   1302608 non-null  int64  
 8   scheduled_arrival_time  1302608 non-null  int64  
 9   temperature             1302608 non-null  float64
 10  precipitation           1302608 non-null  float64
 11  windspeed               1302608 non-null  float64
 12  weathercode             1302608 non-null  int64  
 13  delay                   1302608 non-null  float64
 14  dela

In [68]:
df.describe()

,trip_id,route_id,stop_id,stop_lat,stop_lon,stop_sequence,realtime_arrival_time,scheduled_arrival_time,temperature,precipitation,windspeed,weathercode,delay,delay_previous_stop
count,1.302608e+06,1.302608e+06,1.302608e+06,1.302608e+06,1.302608e+06,1.302608e+06,1.302608e+06,1.302608e+06,1.302608e+06,1.302608e+06,1.302608e+06,1.302608e+06,1.302608e+06,1.302608e+06
mean,2.852078e+08,1.567705e+02,5.487587e+04,4.552688e+01,-7.363510e+01,2.485362e+01,1.745502e+12,1.745501e+12,8.075732e+00,2.719617e-01,1.384666e+01,1.690959e+01,5.394260e+01,5.299962e+01
std,6.364686e+05,1.335764e+02,3.190221e+03,6.389691e-02,8.979414e-02,1.730423e+01,5.699931e+07,5.699617e+07,3.191403e+00,6.967714e-01,5.059409e+00,2.368136e+01,4.188005e+02,4.176206e+02
min,2.847266e+08,1.000000e+01,5.010100e+04,4.540267e+01,-7.395620e+01,1.000000e+00,1.745384e+12,1.745384e+12,-4.000000e-01,0.000000e+00,3.900000e+00,0.000000e+00,-8.377000e+03,-8.377000e+03
25%,2.847768e+08,5.500000e+01,5.219200e+04,4.547651e+01,-7.366915e+01,1.100000e+01,1.745454e+12,1.745454e+12,7.000000e+00,0.000000e+00,9.900000e+00,2.000000e+00,0.000000e+00,0.000000e+00
50%,2.850087e+08,1.210000e+02,5.460900e+04,4.551970e+01,-7.361707e+01,2.200000e+01,1.745511e+12,1.745511e+12,8.300000e+00,0.000000e+00,1.470000e+01,3.000000e+00,0.000000e+00,0.000000e+00
75%,2.852824e+08,1.960000e+02,5.699700e+04,4.557264e+01,-7.357331e+01,3.500000e+01,1.745540e+12,1.745540e+12,1.060000e+01,2.000000e-01,1.820000e+01,5.100000e+01,0.000000e+00,0.000000e+00
max,2.865747e+08,9.680000e+02,6.244200e+04,4.570112e+01,-7.348058e+01,1.170000e+02,1.745595e+12,1.745593e+12,1.350000e+01,3.500000e+00,2.310000e+01,6.300000e+01,3.464900e+04,3.464900e+04


In [70]:
# Export data to CSV
df.to_csv('../data/stm_weather_merged.csv', index=False)

## End